In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor
import sys
sys.path.append('../architectures/')
sys.path.append('../adversarial/')
sys.path.append('../smoothing-adversarial/code/')
from resnet import ResNet18
from core import Smooth
from smoothattacks import PGD_L2

In [2]:
?Smooth

Init signature:
Smooth(
    base_classifier:torch.nn.modules.module.Module,
    num_classes:int,
    sigma:float,
)
Docstring:      A smoothed classifier g 
Init docstring:
:param base_classifier: maps from [batch x channel x height x width] to [batch x num_classes]
:param num_classes:
:param sigma: the noise level hyperparameter
File:           /data/ameya/Projects/StochAT/smoothing-adversarial/code/core.py
Type:           type
Subclasses:     


In [3]:
device = torch.device('cuda')

In [4]:
net = ResNet18()

In [5]:
net = net.to(device)

RuntimeError: cuda runtime error (100) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:47

In [6]:
net.load_state_dict(torch.load('./model-nn-epoch13-robacc78.pt'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [9]:
sm_net = Smooth(net, 10, sigma=0.12)

In [10]:
ds = CIFAR10('../../data/CIFAR10/', download=True, train=False, transform=Compose([ToTensor()]))
dl = DataLoader(ds, batch_size=1)

Files already downloaded and verified


In [11]:
correct = 0.0
for idx,(x, y) in enumerate(dl):
    x = x.to(device)
    y = y.to(device)
    pred = sm_net.predict(x, n=1000, alpha=0.001, batch_size=400)
    correct += y.eq(pred).sum().item()
#     if idx == 1000:
#         break

In [12]:
print(correct)

6346.0


In [ ]:
attacker = PGD_L2(steps=20, device='cuda', max_norm=0.431)

In [ ]:
rob_acc = 0.0
for idx,(x, y) in enumerate(dl):
    x = x.to(device)
    y = y.to(device)
    x = x.repeat((32, 1, 1, 1))
    noise = (1 - int(args.base_attack))* torch.randn_like(x, device='cuda') * 0.12
    x = attacker.attack(net, x, y, 
                        noise=noise, num_noise_vectors=32,
                        no_grad=True,
                        )
    x = x[:1]
    preds = sm_net.predict(x, n=1000, alpha=0.001, batch_size=400)
    rob_acc += y.eq(pred).sum().item()

In [ ]:
print(rob_acc)